# Activity 1
## Cleaning the data

#### Download the dataset
Visit https://archive.ics.uci.edu/ml/datasets/bank+marketing to download the zip file. Unzip the contents to the desired location.

In [31]:
import pandas as pd
bank_data = pd.read_csv('../data/bank.csv', sep=';')

In [2]:
bank_data.head(n=20)

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,no
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,no
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,no
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,no
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,no
5,35,management,single,tertiary,no,747,no,no,cellular,23,feb,141,2,176,3,failure,no
6,36,self-employed,married,tertiary,no,307,yes,no,cellular,14,may,341,1,330,2,other,no
7,39,technician,married,secondary,no,147,yes,no,cellular,6,may,151,2,-1,0,unknown,no
8,41,entrepreneur,married,tertiary,no,221,yes,no,unknown,14,may,57,2,-1,0,unknown,no
9,43,services,married,primary,no,-88,yes,yes,cellular,17,apr,313,1,147,2,failure,no


Numerical columns
- age
- balance
- day
- duration
- campaign
- pdays
- previous

Binary columns
- default
- housing
- loan

Categorical columns to convert to dummy dummy columns
- job
- education
- marital
- contact
- month
- poutcome

Convert 'default' column to binary, rename to 'is_default'

In [6]:
bank_data['default'].value_counts()

no     4445
yes      76
Name: default, dtype: int64

We can see that this column is highly skewed

In [7]:
bank_data['is_default'] = bank_data['default'].apply(lambda row: 1 if row == 'yes' else 0)

Do the same for housing, confirming that there are only two options, yes, and no

In [8]:
bank_data['housing'].value_counts()

yes    2559
no     1962
Name: housing, dtype: int64

In [9]:
bank_data['is_housing'] = bank_data['housing'].apply(lambda row: 1 if row == 'yes' else 0)

Repeat for loan

In [11]:
bank_data['loan'].value_counts()

no     3830
yes     691
Name: loan, dtype: int64

In [12]:
bank_data['is_loan'] = bank_data['loan'].apply(lambda row: 1 if row == 'yes' else 0)

Repeat for marital, however this we will map the value 'married' to 1

In [13]:
bank_data['marital'].value_counts()

married     2797
single      1196
divorced     528
Name: marital, dtype: int64

We can see that there are three options for the marital column, so we will make this a categorical column.

We can do this using the `get_dummies` function in the pandas library

In [45]:
help(pd.get_dummies)

Help on function get_dummies in module pandas.core.reshape.reshape:

get_dummies(data, prefix=None, prefix_sep='_', dummy_na=False, columns=None, sparse=False, drop_first=False)
    Convert categorical variable into dummy/indicator variables
    
    Parameters
    ----------
    data : array-like, Series, or DataFrame
    prefix : string, list of strings, or dict of strings, default None
        String to append DataFrame column names
        Pass a list with length equal to the number of columns
        when calling get_dummies on a DataFrame. Alternativly, `prefix`
        can be a dictionary mapping column names to prefixes.
    prefix_sep : string, default '_'
        If appending prefix, separator/delimiter to use. Or pass a
        list or dictionary as with `prefix.`
    dummy_na : bool, default False
        Add a column to indicate NaNs, if False NaNs are ignored.
    columns : list-like, default None
        Column names in the DataFrame to be encoded.
        If `columns` i

In [23]:
marital_dummies = pd.get_dummies(bank_data['marital'])
marital_dummies.head(n=20)

,divorced,married,single
0,0,1,0
1,0,1,0
2,0,0,1
3,0,1,0
4,0,1,0
5,0,0,1
6,0,1,0
7,0,1,0
8,0,1,0
9,0,1,0


We can see that there is a bit of redundant information here, we have three total options, but we know that whenever two of the columns are zero, the other column HAS to be 1, since everyone has to fall into one of the three options. We can then drop one of the columns and assume that a zero in the remaining columns means that the dropped column is true.

Here we will drop the `divorced` column and change the name of the remaining columns from `married` to `is_married` and from `single` to `is_single` for consistency and clarity.

In [24]:
marital_dummies.drop('divorced', axis=1, inplace=True)
marital_dummies.columns = [f'is_{colname}' for colname in marital_dummies.columns]

In [25]:
marital_dummies.head()

,is_married,is_single
0,1,0
1,1,0
2,0,1
3,1,0
4,1,0


Now we will join it back to the original dataset

In [34]:
bank_data = pd.concat([bank_data, marital_dummies], axis=1)

Let's do the same for the `job`, `education`, and `contact`

In [35]:
bank_data['job'].value_counts()

management       969
blue-collar      946
technician       768
admin.           478
services         417
retired          230
self-employed    183
entrepreneur     168
unemployed       128
housemaid        112
student           84
unknown           38
Name: job, dtype: int64

The `unknown` value seems most appropriate to drop here

In [36]:
job_dummies = pd.get_dummies(bank_data['job'])
job_dummies.drop('unknown', axis=1, inplace=True)

Let's also rename the columns so we know they came from the job field

In [37]:
job_dummies.columns = [f'job_{colname}' for colname in job_dummies.columns]

In [38]:
bank_data = pd.concat([bank_data, job_dummies], axis=1)

In [39]:
bank_data['education'].value_counts()

secondary    2306
tertiary     1350
primary       678
unknown       187
Name: education, dtype: int64

In [40]:
edu_dummies = pd.get_dummies(bank_data['education'])
edu_dummies.drop('unknown', axis=1, inplace=True)
edu_dummies.columns = [f'education_{colname}' for colname in edu_dummies.columns]

In [41]:
bank_data = pd.concat([bank_data, edu_dummies], axis=1)

And for the contact field

In [42]:
bank_data['contact'].value_counts()

cellular     2896
unknown      1324
telephone     301
Name: contact, dtype: int64

In [43]:
contact_dummies = pd.get_dummies(bank_data['contact'])
contact_dummies.drop('unknown', axis=1, inplace=True)
contact_dummies.columns = [f'contact_{colname}' for colname in contact_dummies.columns]
bank_data = pd.concat([bank_data, contact_dummies], axis=1)

For the `month` field we may want to do something a bit different because there is an inherent order to the months. In this case we may want to label the months in the chronological order. 

To do this we can create a map to represent the months as numbers and preserve the ordinality.

In [44]:
month_map = {'jan':1, 'feb':2, 'mar':3, 'apr':4, 'may':5, 'jun':6, 'jul':7, 'aug':8, 'sep':9, 'oct':10, 'nov':11, 'dec': 12}

In [47]:
bank_data['month'].value_counts()

may    1398
jul     706
aug     633
jun     531
nov     389
apr     293
feb     222
jan     148
oct      80
sep      52
mar      49
dec      20
Name: month, dtype: int64

Whoa! The month is highly skewed, I wonder why there are so many entries in May?

In [ ]:
bank_data['month'] = bank_data['month'].map(month_map)

In [9]:
# create features table
X = bank_data.drop('y', axis=1)
print(f'features table has {X.shape[0]} rows and {X.shape[1]} columns')

features dataset has 4521 rows and 16 columns


In [11]:
y = bank_data['y']
print(f'target table has {y.shape[0]} rows')

target table has 4521 rows
